In [ ]:
import networkx as nx
from pathlib import Path

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from flinspect.gen_dependency_graph import gen_module_dependency_graph, gen_call_graph

# Generate Call Graph

# Plot Module Dependency Graph

In [ ]:
build_dirs = [
    "/glade/work/altuntas/turbo-stack/bin/flang_ptree/FMS/",
    #"/glade/work/altuntas/turbo-stack/bin/flang_ptree/MOM6"
]

ptree_files = [f for dir in build_dirs for f in Path(dir).glob("*_ptree")]

In [ ]:
G = gen_module_dependency_graph(ptree_files)
assert nx.is_directed_acyclic_graph(G)

In [ ]:
# Determine source paths via the path_names files in build dirs
# This will be used to color nodes and edges based on where modules are located
path_names = {}
for build_dir in build_dirs:
    path_names_file = Path(build_dir)/"path_names"
    with open(path_names_file) as f:
        for line in f:
            src_file_path = Path(line.strip())
            path_names[src_file_path.stem.lower()] = src_file_path.as_posix()

In [ ]:
def get_color(G, node):
    source_name = G.nodes[node].get('source_name','')
    source_path = path_names.get(source_name.lower(),'')
    
    node_color, edge_color = "grey", "lightgrey"
    if "/config_src/infra" in source_path:
        node_color, edge_color = "darkred", "red"    
    elif "/MARBL/" in source_path:
        node_color, edge_color = "purple", "orchid"
    elif "/CVMix-src/" in source_path:
        node_color, edge_color = "chocolate", "sandybrown"
    elif "/GSW-Fortran/" in source_path:
        node_color, edge_color = "royalblue", "cornflowerblue"
    elif "/MOM6/" in source_path:
        node_color, edge_color = "lightseagreen", "turquoise" # for core MOM6        
    elif "/FMS/" in source_path:
        node_color, edge_color = "orange", "gold"
        
    return node_color, edge_color

In [ ]:
# Determine Node positions using 'fdp' layout
pos = nx.nx_agraph.graphviz_layout(G, prog='fdp')

In [ ]:
from pyvis.network import Network

In [ ]:
net = Network(notebook=True, directed=True, height='800px', cdn_resources='in_line')

net.set_options("""
var options = {
  "physics": { "enabled": false },
  "nodes": { "shape": "dot", "size": 10 },
  "edges": { "arrows": { "to": { "enabled": true, "scaleFactor": 0.3 }    } }
}
""")

# Add nodes with positions
for node in G.nodes():
    node_color, _ = get_color(G, node)
    x, y = pos[node]
    net.add_node(str(node), color=node_color, label=str(node), x=x, y=y, fixed=True)  # y flipped for browser coord system

# Add edges
for source, target in G.edges():
    _, edge_color = get_color(G, source)
    net.add_edge(str(source), str(target), color=edge_color)


In [ ]:
# 3232 or 3164
len(G.edges)

In [ ]:
# net.show("module_dependency.html")

In [ ]:
from flinspect.parse_tree_parser import *
from flinspect.utils import *

In [ ]:
len(Subroutine._registry)

In [ ]:
len(Function._registry)

In [ ]:
Module('mpp_mod')

In [ ]:
Module('mpp_modx')

In [ ]:
Function._registry['field_manager_mod::dump_list']

# Plot Call Graph

In [ ]:
G = gen_call_graph(ptree_files);

In [ ]:
from flinspect.parse_tree_node import Callable

In [ ]:
import gc
num_edges = 0
for obj in gc.get_objects():
    if isinstance(obj, Callable):
        if obj.callees:
            callee_names = [callee.name for callee in obj.callees]
            if num_edges < (cutoff := 20):
                print(f"caller: {obj}, callee: {callee_names}")
            elif num_edges == cutoff:
                print("...")
        
            num_edges += len(callee_names)

print(f"\nTotal edges (i.e., calls) in call graph: {num_edges}")